# Overview

## Import package

In [2]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import sys

## Constant Value

In [183]:
df_cast_crew = pd.read_csv('credits.csv')

In [184]:
df_cast_crew

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


# Feature Observation

## Credits

### Consists of Cast and Crew Information for all our movies. 

### 1. Cast: Information about Cast in movie


#### (1) cast_id
#### (2) character: Name of character
#### (3) credit_id
#### (4) gender: category datatype. 1=female 2=male
#### (5) id
#### (6) name: Name of cast
#### (7) order
#### (8) profile_path

### 2. Crew: Staff group information in movies

#### (1) credit_id
#### (2) department: Employee's department: Directing,Writing,Crew, etc
#### (3) gender: Employee's gender
#### (4) id
#### (5) job: employee's job title
#### (6) name
#### (7) profile_path

## Movie Metadata

### The main Movies Metadata file. Contains information on 45,000 movies featured in the Full MovieLens dataset. Features include posters, backdrops, budget, revenue, release dates, languages, production countries and companies.

In [398]:
df_movie=pd.read_csv('movies_metadata.csv',low_memory=False)
pd.set_option('display.max_columns', None)

In [399]:
df_movie

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45

## Keywords

### Contains the movie plot keywords for our MovieLens movies. 

In [ ]:
df_keyword=pd.read_csv('keywords.csv')
pd.set_option('display.max_columns', None)

In [ ]:
df_keyword

## Ratings_small

### The subset of 100,000 ratings from 700 users on 9,000 movies.

In [196]:
df_rate = pd.read_csv('ratings_small.csv')

In [197]:
df_rate

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


## Links

In [198]:
df_links = pd.read_csv('links.csv')
pd.set_option('display.max_columns', None)

In [199]:
df_links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
45838,176269,6209470,439050.0
45839,176271,2028550,111109.0
45840,176273,303758,67758.0
45841,176275,8536,227506.0


# #####Data Cleaning

## Cast('credit.csv')

In [185]:
df_cast = df_cast_crew[['cast','id']]

In [186]:
df_cast

,cast,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...",11862
...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...",227506


In [266]:
id = df_cast['id']
# print(id)
split_1 = df_cast['cast'].str.split('},')
data = []

for index, list in enumerate(split_1):
    if list != ['[]']:
        for split in list:
            split_2 = split.split(", '") 
            # print(split_2[1])
            new_row = []
            new_row.append(id[index])
        
            for small_split in split_2:
                split_3 = small_split.split(':')
                # print(split_3)
                # print(split_3[1])
                if len(split_3) == 2:
                    new_row.append(split_3[1]) 
    #             # print(new_row)
            data.append(new_row)
            # print(new_row)
df_cast_real = pd.DataFrame(data, columns=['id', 'cast_id', 'character','credit_id', 'gender', 'unknown_id', 'name', 'order', 'profile_path'])
df_cast_real

,id,cast_id,character,credit_id,gender,unknown_id,name,order,profile_path
0,862,14,'Woody (voice)','52fe4284c3a36847f8024f95',2,31,'Tom Hanks',0,'/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'
1,862,15,'Buzz Lightyear (voice)','52fe4284c3a36847f8024f99',2,12898,'Tim Allen',1,'/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'
2,862,16,'Mr. Potato Head (voice)','52fe4284c3a36847f8024f9d',2,7167,'Don Rickles',2,'/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'
3,862,17,'Slinky Dog (voice)','52fe4284c3a36847f8024fa1',2,12899,'Jim Varney',3,'/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'
4,862,18,'Rex (voice)','52fe4284c3a36847f8024fa5',2,12900,'Wallace Shawn',4,'/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg'
...,...,...,...,...,...,...,...,...,...
562469,227506,2,'','52fe4ea59251416c7515d7d5',2,544742,'Iwan Mosschuchin',0,None
562470,227506,3,'','52fe4ea59251416c7515d7d9',1,1090923,'Nathalie Lissenko',1,None
562471,227506,4,'','52fe4ea59251416c7515d7dd',2,1136422,'Pavel Pavlov',2,None
562472,227506,5,'','52fe4ea59251416c7515d7e1',0,1261758,'Aleksandr Chabrov',3,None


In [350]:
df_movie

,adult,belongs_to_collection,budget,genres,homepage,movie_id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,

In [351]:
df_movie['belongs_to_collection'][2]

"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}"

In [352]:
df_movie['genres'][2]

"[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]"

In [353]:
df_movie.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
movie_id                     0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

## Crew('credit.csv')

In [609]:
df_crew = df_cast_crew[['crew','id']]

In [610]:
df_crew

,crew,id
0,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...
45471,"[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [689]:
movie_id = df_cast['id']
# print(id)
data_crew = []

In [686]:
split_1 = df_crew['crew'].str.split('},')

In [688]:
split_1[0]

["[{'credit_id': '52fe4284c3a36847f8024f49', 'department': 'Directing', 'gender': 2, 'id': 7879, 'job': 'Director', 'name': 'John Lasseter', 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'",
 " {'credit_id': '52fe4284c3a36847f8024f4f', 'department': 'Writing', 'gender': 2, 'id': 12891, 'job': 'Screenplay', 'name': 'Joss Whedon', 'profile_path': '/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg'",
 " {'credit_id': '52fe4284c3a36847f8024f55', 'department': 'Writing', 'gender': 2, 'id': 7, 'job': 'Screenplay', 'name': 'Andrew Stanton', 'profile_path': '/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg'",
 " {'credit_id': '52fe4284c3a36847f8024f5b', 'department': 'Writing', 'gender': 2, 'id': 12892, 'job': 'Screenplay', 'name': 'Joel Cohen', 'profile_path': '/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg'",
 " {'credit_id': '52fe4284c3a36847f8024f61', 'department': 'Writing', 'gender': 0, 'id': 12893, 'job': 'Screenplay', 'name': 'Alec Sokolow', 'profile_path': '/v79vlRYi94BZUQnkkyznbGUZLjT.jpg'",
 " {'credit_id': '52fe4284c3a36847f

In [690]:
for index, list in enumerate(split_1):
    if list != ['[]']:
        for split in list:
            split_2 = split.split(", '") 
            # print(split_2[1])
            new_row = []
            new_row.append(movie_id[index])
        
            for small_split in split_2:
                split_3 = small_split.split(':')
                # print(split_3)
                # print(split_3[1])
                if len(split_3) == 2:
                    new_row.append(split_3[1]) 
    #             # print(new_row)
            data_crew.append(new_row)
            # print(new_row)
df_cast_real = pd.DataFrame(data_crew, columns=['movie_id', 'credit_id', 'department','gender', 'unknown_id', 'job', 'name', 'profile_path'])
df_cast_real

,movie_id,credit_id,department,gender,unknown_id,job,name,profile_path
0,862,'52fe4284c3a36847f8024f49','Directing',2,7879,'Director','John Lasseter','/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'
1,862,'52fe4284c3a36847f8024f4f','Writing',2,12891,'Screenplay','Joss Whedon','/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg'
2,862,'52fe4284c3a36847f8024f55','Writing',2,7,'Screenplay','Andrew Stanton','/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg'
3,862,'52fe4284c3a36847f8024f5b','Writing',2,12892,'Screenplay','Joel Cohen','/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg'
4,862,'52fe4284c3a36847f8024f61','Writing',0,12893,'Screenplay','Alec Sokolow','/v79vlRYi94BZUQnkkyznbGUZLjT.jpg'
...,...,...,...,...,...,...,...,...
464309,67758,'52fe4776c3a368484e0c8399','Sound',0,549356,'Original Music Composer','Richard McHugh',None
464310,67758,'52fe4776c3a368484e0c839f','Camera',2,58818,'Director of Photography','João Fernandes',None}]
464311,227506,'533bccebc3a36844cf0011a7','Directing',0,1085341,'Director','Yakov Protazanov','/yyjbGdCs2ZN6IlZNCfmBWyuRDlt.jpg'
464312,227506,'58ebbc26925141281908aa0a','Production',2,1195656,'Producer','Joseph N. Ermolieff',None}]


In [ ]:
df_cast_real['credit_id'] = df_cast_real['credit_id'].str.replace("'", "", regex=False)
df_cast_real['department'] = df_cast_real['department'].str.replace("'", "", regex=False)
df_cast_real['job'] = df_cast_real['job'].str.replace("'", "", regex=False)
df_cast_real['name'] = df_cast_real['name'].str.replace("'", "", regex=False)
df_cast_real['name'] = df_cast_real['name'].str.replace("'", "", regex=False)

## Genres ('movie_metadata.csv')

In [373]:
df_genres = df_movie[['genres','id']]

In [374]:
df_genres

,genres,id
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357
4,"[{'id': 35, 'name': 'Comedy'}]",11862
...,...,...
45461,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",439050
45462,"[{'id': 18, 'name': 'Drama'}]",111109
45463,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758
45464,[],227506


In [375]:
split = df_genres['genres'].str.split(', {')

In [376]:
data_genres = []
moive_id = df_movie['id']

In [391]:
for index, item in enumerate(split):
    # print(item)
    if item !=['[]']:
        # print(item)
        for split_1 in item:
            # print(split_1)
            split_2 = split_1.split(',')
            # print(split_2)
            new_row = []
            new_row.append(moive_id[index])

            # print(split_2)
            for split_3 in split_2:
                # print(split_3)
                split_4 = split_3.split(':')
                # print(split_4[1])
                new_row.append(split_4[1])
                # print(new_row)
                data_genres.append(new_row)
                
df_genres_real = pd.DataFrame(data_genres, columns=['movie_id', 'id', 'name'])  
df_genres_real

,movie_id,id,name
0,862,16,'Animation'}
1,862,16,'Animation'}
2,862,35,'Comedy'}
3,862,35,'Comedy'}
4,862,10751,'Family'}]
...,...,...,...
1093267,67758,28,'Action'}
1093268,67758,18,'Drama'}
1093269,67758,18,'Drama'}
1093270,67758,53,'Thriller'}]


In [392]:
df_genres_real['name'] = df_genres_real['name'].str.replace("}", "", regex=False)
df_genres_real['name'] = df_genres_real['name'].str.replace("'", "", regex=False)
df_genres_real['name'] = df_genres_real['name'].str.replace("]", "", regex=False)

In [393]:
df_genres_real

,movie_id,id,name
0,862,16,Animation
1,862,16,Animation
2,862,35,Comedy
3,862,35,Comedy
4,862,10751,Family
...,...,...,...
1093267,67758,28,Action
1093268,67758,18,Drama
1093269,67758,18,Drama
1093270,67758,53,Thriller


## Collection ('movie_metadata.csv')

In [440]:
df_collection = df_movie[['belongs_to_collection','id']].copy()
df_collection['belongs_to_collection'].fillna('unknown', inplace=True)

In [533]:
df_collection

,belongs_to_collection,id
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",862
1,unknown,8844
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",15602
3,unknown,31357
4,"{'id': 96871, 'name': 'Father of the Bride Col...",11862
...,...,...
45461,unknown,439050
45462,unknown,111109
45463,unknown,67758
45464,unknown,227506


In [534]:
df_collection['belongs_to_collection'][0]

"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"

In [442]:
split = df_collection['belongs_to_collection'].str.split(',')

In [459]:
data_collection = []
moive_id = df_movie['id']

for index, item in enumerate(split):
    new_row = []
    new_row.append(moive_id[index])
    # print(item)
    if item != 'unknown':
        # print(item)
        for split_1 in item:
            # print(split_1)
            split_2 = split_1.split(':')
            # print(split_2)

            if len(split_2) == 2:
                new_row.append(split_2[1])
                # print(new_row)
                data_collection.append(new_row)
                
df_collection_real = pd.DataFrame(data_collection, columns=['movie_id', 'id', 'collection_name', 'poster_path', 'backdrop_path'])  
df_collection_real

,movie_id,id,collection_name,poster_path,backdrop_path
0,862,10194,'Toy Story Collection','/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg','/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}
1,862,10194,'Toy Story Collection','/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg','/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}
2,862,10194,'Toy Story Collection','/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg','/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}
3,862,10194,'Toy Story Collection','/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg','/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}
4,15602,119050,'Grumpy Old Men Collection','/nLvUdqgPgm3F85NMCii9gVFUcet.jpg','/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}
...,...,...,...,...,...
17860,460135,477208,'DC Super Hero Girls Collection',None,None}
17861,182030,200641,'Red Lotus Collection','/yf9Eod9ANXyHTzDpAxz9ezgvlL4.jpg','/3fhHbLeO3DqdHvgHg5szs399eBb.jpg'}
17862,182030,200641,'Red Lotus Collection','/yf9Eod9ANXyHTzDpAxz9ezgvlL4.jpg','/3fhHbLeO3DqdHvgHg5szs399eBb.jpg'}
17863,182030,200641,'Red Lotus Collection','/yf9Eod9ANXyHTzDpAxz9ezgvlL4.jpg','/3fhHbLeO3DqdHvgHg5szs399eBb.jpg'}


In [463]:
df_collection_real['collection_name'] = df_collection_real['collection_name'].str.replace("'", "", regex=False)
df_collection_real['poster_path'] = df_collection_real['poster_path'].str.replace("'", "", regex=False)
df_collection_real['backdrop_path'] = df_collection_real['backdrop_path'].str.replace("'", "", regex=False)
df_collection_real['backdrop_path'] = df_collection_real['backdrop_path'].str.replace("}", "", regex=False)

In [464]:
df_collection_real

,movie_id,id,collection_name,poster_path,backdrop_path
0,862,10194,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,862,10194,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
2,862,10194,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
3,862,10194,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
4,15602,119050,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
...,...,...,...,...,...
17860,460135,477208,DC Super Hero Girls Collection,None,None
17861,182030,200641,Red Lotus Collection,/yf9Eod9ANXyHTzDpAxz9ezgvlL4.jpg,/3fhHbLeO3DqdHvgHg5szs399eBb.jpg
17862,182030,200641,Red Lotus Collection,/yf9Eod9ANXyHTzDpAxz9ezgvlL4.jpg,/3fhHbLeO3DqdHvgHg5szs399eBb.jpg
17863,182030,200641,Red Lotus Collection,/yf9Eod9ANXyHTzDpAxz9ezgvlL4.jpg,/3fhHbLeO3DqdHvgHg5szs399eBb.jpg


In [535]:
# remove the duplicate row
df_collection_real = df_collection_real.drop_duplicates()

In [536]:
df_collection_real

,movie_id,id,collection_name,poster_path,backdrop_path
0,862,10194,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
4,15602,119050,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
8,11862,96871,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg
12,710,645,James Bond Collection,/HORpg5CSkmeQlAolx3bKMrKgfi.jpg,/6VcVl48kNKvdXOZfJPdarlUGOsk.jpg
16,21032,117693,Balto Collection,/w0ZgH6Lgxt2bQYnf1ss74UvYftm.jpg,/9VM5LiJV0bGb1st1KyHA3cVnO2G.jpg
...,...,...,...,...,...
17845,24568,37261,The Carry On Collection,/2P0HNrYgKDvirV8RCdT1rBSJdbJ.jpg,/38tF1LJN7ULeZAuAfP7beaPMfcl.jpg
17849,19307,37261,The Carry On Collection,/2P0HNrYgKDvirV8RCdT1rBSJdbJ.jpg,/38tF1LJN7ULeZAuAfP7beaPMfcl.jpg
17853,21251,37261,The Carry On Collection,/2P0HNrYgKDvirV8RCdT1rBSJdbJ.jpg,/38tF1LJN7ULeZAuAfP7beaPMfcl.jpg
17857,460135,477208,DC Super Hero Girls Collection,None,None


## Production companies ('movie_metadata.csv')

In [625]:
df_company = df_movie[['id', 'production_companies']].copy()

In [626]:
df_company['production_companies'].isnull().sum()

3

In [627]:
df_company['production_companies'].fillna('unknown', inplace=True)

In [628]:
df_company['production_companies'].isnull().sum()

0

In [629]:
split = df_company['production_companies'].str.split(', {')

In [630]:
split[0]

["[{'name': 'Pixar Animation Studios', 'id': 3}]"]

In [631]:
data_company = []
moive_id = df_movie['id']

for index, item in enumerate(split):
    # print(item)
    if item !=['[]'] and item !='unknown':
        # print(item)
        for split_1 in item:
            # print(split_1)
            split_2 = split_1.split(',')
            # print(split_2)
            new_row = []
            new_row.append(moive_id[index])

            # print(split_2)
            for split_3 in split_2:
                # print(split_3)
                split_4 = split_3.split(':')
                if len(split_4) == 2:
                    # print(split_4[1])
                    new_row.append(split_4[1])
                    # print(new_row)
                    data_company.append(new_row)
                
df_company_real = pd.DataFrame(data_company, columns=['movie_id', 'company_name', 'id'])  
df_company_real

,movie_id,company_name,id
0,862,'Pixar Animation Studios',3}]
1,862,'Pixar Animation Studios',3}]
2,8844,'TriStar Pictures',559}
3,8844,'TriStar Pictures',559}
4,8844,'Teitler Film',2550}
...,...,...,...
141076,111109,'Sine Olivia',19653}]
141077,67758,'American World Pictures',6165}]
141078,67758,'American World Pictures',6165}]
141079,227506,'Yermoliev',88753}]


In [529]:
df_company_real['company_name'] = df_company_real['company_name'].str.replace("'", "", regex=False)
df_company_real['id'] = df_company_real['id'].str.replace("}]", "", regex=False)
df_company_real['id'] = df_company_real['id'].str.replace("}", "", regex=False)

In [531]:
# remove the duplicate row
df_company_real = df_company_real.drop_duplicates()

In [532]:
df_company_real

,movie_id,company_name,id
0,862,Pixar Animation Studios,3
2,8844,TriStar Pictures,559
4,8844,Teitler Film,2550
6,8844,Interscope Communications,10201
8,15602,Warner Bros.,6194
...,...,...,...
141071,30840,20th Century Fox Television,16323
141073,30840,CanWest Global Communications,38978
141075,111109,Sine Olivia,19653
141077,67758,American World Pictures,6165


## Production_countries ('movie_metadata.csv')

In [646]:
df_country = df_movie[['id','production_countries']].copy()
df_country['production_countries'][0]

"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"

In [647]:
df_country['production_countries'].isnull().sum()

3

In [648]:
df_country['production_countries'].fillna('unknown', inplace=True)
df_country['production_countries'].isnull().sum()

0

In [658]:
df_country['production_countries']

0        [{'iso_3166_1': 'US', 'name': 'United States o...
1        [{'iso_3166_1': 'US', 'name': 'United States o...
2        [{'iso_3166_1': 'US', 'name': 'United States o...
3        [{'iso_3166_1': 'US', 'name': 'United States o...
4        [{'iso_3166_1': 'US', 'name': 'United States o...
                               ...                        
45461               [{'iso_3166_1': 'IR', 'name': 'Iran'}]
45462        [{'iso_3166_1': 'PH', 'name': 'Philippines'}]
45463    [{'iso_3166_1': 'US', 'name': 'United States o...
45464             [{'iso_3166_1': 'RU', 'name': 'Russia'}]
45465     [{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]
Name: production_countries, Length: 45466, dtype: object

In [663]:
split = df_country['production_countries'].str.split(', {')

In [664]:
split[11]

["[{'iso_3166_1': 'FR', 'name': 'France'}",
 "'iso_3166_1': 'US', 'name': 'United States of America'}]"]

In [665]:
data_country = []
moive_id = df_movie['id']

for index, item in enumerate(split):
    # print(item)
    if item !=['[]'] and item !='unknown':
        # print(item)
        for split_1 in item:
            # print(split_1)
            split_2 = split_1.split(',')
            # print(split_2)
            new_row = []
            new_row.append(moive_id[index])
            
            # print(new_row)

            # print(split_2)
            for split_3 in split_2:
                # print(split_3)
                split_4 = split_3.split(':')
                # print(split_4)
                
                if len(split_4) == 2:
                    # print(split_4[1])
                    new_row.append(split_4[1])
                    # print(new_row)
                    data_country.append(new_row)
# print(data_country)                
data_country_real = pd.DataFrame(data_country, columns=['movie_id', 'iso_3166_1', 'country_name'])  
data_country_real


,movie_id,iso_3166_1,country_name
0,862,'US','United States of America'}]
1,862,'US','United States of America'}]
2,8844,'US','United States of America'}]
3,8844,'US','United States of America'}]
4,15602,'US','United States of America'}]
...,...,...,...
98841,67758,'US','United States of America'}]
98842,227506,'RU','Russia'}]
98843,227506,'RU','Russia'}]
98844,461257,'GB','United Kingdom'}]


In [666]:
data_country_real['iso_3166_1'] = data_country_real['iso_3166_1'].str.replace("'", "", regex=False)
data_country_real['country_name'] = data_country_real['country_name'].str.replace("'}]", "", regex=False)
data_country_real['country_name'] = data_country_real['country_name'].str.replace("'", "", regex=False)
data_country_real = data_country_real.drop_duplicates()

In [669]:
data_country_real

,movie_id,iso_3166_1,country_name
0,862,US,United States of America
2,8844,US,United States of America
4,15602,US,United States of America
6,31357,US,United States of America
8,11862,US,United States of America
...,...,...,...
98836,439050,IR,Iran
98838,111109,PH,Philippines
98840,67758,US,United States of America
98842,227506,RU,Russia


## Spoken languages ('movie_metadata.csv')

In [677]:
df_language = df_movie[['spoken_languages','id']].copy()

In [678]:
df_language

,spoken_languages,id
0,"[{'iso_639_1': 'en', 'name': 'English'}]",862
1,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",8844
2,"[{'iso_639_1': 'en', 'name': 'English'}]",15602
3,"[{'iso_639_1': 'en', 'name': 'English'}]",31357
4,"[{'iso_639_1': 'en', 'name': 'English'}]",11862
...,...,...
45461,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",439050
45462,"[{'iso_639_1': 'tl', 'name': ''}]",111109
45463,"[{'iso_639_1': 'en', 'name': 'English'}]",67758
45464,[],227506


In [679]:
df_language['spoken_languages'].isnull().sum()

6

In [680]:
df_language['spoken_languages'].fillna('unknown', inplace=True)
df_language['spoken_languages'].isnull().sum()

0

In [681]:
data_language = []
moive_id = df_movie['id']

for index, item in enumerate(split):
    # print(item)
    if item !=['[]'] and item !='unknown':
        # print(item)
        for split_1 in item:
            # print(split_1)
            split_2 = split_1.split(',')
            # print(split_2)
            new_row = []
            new_row.append(moive_id[index])
            
            # print(new_row)

            # print(split_2)
            for split_3 in split_2:
                # print(split_3)
                split_4 = split_3.split(':')
                # print(split_4)
                
                if len(split_4) == 2:
                    # print(split_4[1])
                    new_row.append(split_4[1])
                    # print(new_row)
                    data_language.append(new_row)
# print(data_country)                
data_language_real = pd.DataFrame(data_language, columns=['movie_id', 'iso_639_1', 'name'])  
data_language_real


,movie_id,iso_639_1,name
0,862,'US','United States of America'}]
1,862,'US','United States of America'}]
2,8844,'US','United States of America'}]
3,8844,'US','United States of America'}]
4,15602,'US','United States of America'}]
...,...,...,...
98841,67758,'US','United States of America'}]
98842,227506,'RU','Russia'}]
98843,227506,'RU','Russia'}]
98844,461257,'GB','United Kingdom'}]


In [682]:
data_language_real['iso_639_1'] = data_language_real['iso_639_1'].str.replace("'", "", regex=False)
data_language_real['name'] = data_language_real['name'].str.replace("'}]", "", regex=False)
data_language_real['name'] = data_language_real['name'].str.replace("'", "", regex=False)
data_language_real = data_language_real.drop_duplicates()

In [683]:
data_language_real

,movie_id,iso_639_1,name
0,862,US,United States of America
2,8844,US,United States of America
4,15602,US,United States of America
6,31357,US,United States of America
8,11862,US,United States of America
...,...,...,...
98836,439050,IR,Iran
98838,111109,PH,Philippines
98840,67758,US,United States of America
98842,227506,RU,Russia


# Use Case

## 1. 电影的基本信息（题材，语言，名字，时长，发行国家，发行商，各种basic information）
## 2. 人们对电影的感兴趣程度（popularity, rating, keyword, 男女演员的受欢迎程度）
## 3. 电影的发行情况（是否能成功发行，不同题材的popularity, revenue，哪些演员更受欢迎，评分）

### Audiences and distributors have different ways of evaluating movies. When it comes to choosing a movie to watch during leisure time, people are more concerned about the genre of the movie, whether it features their favorite actors, and how popular and well-rated it is. On the other hand, for distributors, they are more interested in understanding the annual quantity of movies, as well as the investment and profitability of each film.

## 2. TO AUDIENCE（the point is 哪种因素更加影响人们对电影的喜爱程度 -> 哪种电影更受人们欢迎）

## 1. What is the top 10 movie genres

## 2. What is the top 100 popular movie? 

## 3. Who are the most frequent male and female actors appearing in the top 100 popular movies ?

## 4. Which kind of language is more popular in movie

## 6. Which genre of movies is more popular among people (has higher ratings)?

## TO Distributor（哪种类型的电影很难发行，哪种电影商业价值最高，哪些演员商业价值更高）

## 1. Which genre of movies has higher profits?

## 2. What are the top 100 movies in terms of audience count, and what are the top 100 movies in terms of profit?

## 3. Who is the actor/actress with the highest appearance frequency among the top 100 movies in terms of profit?

## 4. Which genres have a lower distribution rate?

In [ ]:
### ER Diagram 
 